In [1]:
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict

# 配置参数
VIDEO_PATH = "highway_d.avi"          # 输入视频路径
OUTPUT_PATH = "output_video_d.mp4"    # 输出视频路径
COUNT_LINE_Y = 760                  # 计数线位置（垂直方向y坐标）
CLASS_NAMES = [2, 5, 7]             # 车辆类别ID（COCO: car=2, bus=5, truck=7）
TRACKER_CONFIG = "botsort.yaml"     # 跟踪算法配置文件（可选：botsort.yaml, bytetrack.yaml）

def main():
    # 加载模型
    model = YOLO("yolov8l.pt")
    
    # 打开视频
    cap = cv2.VideoCapture(VIDEO_PATH)
    assert cap.isOpened(), "视频打开失败"
    
    # 获取视频信息
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # 初始化视频写入
    writer = cv2.VideoWriter(OUTPUT_PATH, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
    
    # 记录车辆轨迹和计数
    track_history = defaultdict(list)
    counted_ids = set()
    total_count = 0

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break

        # 使用跟踪模式推理
        results = model.track(
            frame,
            persist=True,
            tracker=TRACKER_CONFIG,
            classes=CLASS_NAMES,
            verbose=False  # 关闭冗余输出
        )

        # 获取当前帧的检测结果
        if results[0].boxes.id is not None:
            boxes = results[0].boxes.xyxy.cpu().numpy()
            track_ids = results[0].boxes.id.cpu().numpy().astype(int)
            class_ids = results[0].boxes.cls.cpu().numpy().astype(int)

            # 遍历每个检测目标
            for box, track_id, class_id in zip(boxes, track_ids, class_ids):
                x1, y1, x2, y2 = box
                center_x = int((x1 + x2) / 2)
                center_y = int((y1 + y2) / 2)

                # 记录轨迹（仅保存最近30个点）
                track = track_history[track_id]
                track.append((center_x, center_y))
                if len(track) > 30:
                    track.pop(0)

                # 判断是否跨越计数线
                if len(track) >= 2:
                    prev_y = track[-2][1]
                    curr_y = center_y
                    
                    # 从上方进入下方：计数线检测
                    if prev_y < COUNT_LINE_Y and curr_y >= COUNT_LINE_Y and track_id not in counted_ids:
                        total_count += 1
                        counted_ids.add(track_id)

                # 绘制检测框和ID
                color = (0, 255, 0)
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
                cv2.putText(frame, f"ID: {track_id}", (int(x1), int(y1)-10), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # 绘制计数线和统计信息
        cv2.line(frame, (0, COUNT_LINE_Y), (width, COUNT_LINE_Y), (0, 0, 255), 3)
        cv2.putText(frame, f"Total Vehicles: {total_count}", (20, 40), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # 写入视频帧
        writer.write(frame)
        
        # 实时显示（按ESC退出）
        cv2.imshow("Vehicle Counting", frame)
        if cv2.waitKey(1) == 27:
            break

    # 释放资源
    cap.release()
    writer.release()
    cv2.destroyAllWindows()
    print(f"计数完成！总车辆数: {total_count}")

if __name__ == "__main__":
    main()

计数完成！总车辆数: 49
